In [ ]:
## MdB-Stammdaten in Python 

In [1]:
import requests
import zipfile
import io

# URL der Datei
url = "https://www.bundestag.de/resource/blob/472878/56d0514de78abb261d81ab940b9deed7/MdB-Stammdaten.zip"

# Herunterladen der Datei
response = requests.get(url)

# Überprüfen, ob der Download erfolgreich war
if response.status_code == 200:
    # Zip-Datei im Speicher öffnen
    with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
        # Extrahieren der Dateien in das aktuelle Verzeichnis
        zip_ref.extractall("MdB_Stammdaten")
    print("Datei erfolgreich heruntergeladen und entpackt.")
else:
    print(f"Fehler beim Herunterladen der Datei: Statuscode {response.status_code}")


Datei erfolgreich heruntergeladen und entpackt.


In [1]:
import xml.etree.ElementTree as ET

# Pfad zur XML-Datei
xml_file = './MdB_Stammdaten/MDB_STAMMDATEN.XML'

# XML-Datei einlesen und parsen
tree = ET.parse(xml_file)
root = tree.getroot()

# Liste zur Speicherung aktiver Abgeordneter 2024
active_mdbs = []

# Aktuelles Jahr
current_year_start = '01.01.2024'
current_year_end = '31.12.2024'

# Abgeordnete durchlaufen
for mdb in root.findall('MDB'):
    # Biografische Angaben und Wahlperioden extrahieren
    bio_info = mdb.find('BIOGRAFISCHE_ANGABEN')
    wahlperioden = mdb.find('WAHLPERIODEN')

    # Sicherstellen, dass Wahlperioden vorhanden sind
    if wahlperioden is not None:
        for wahlperiode in wahlperioden.findall('WAHLPERIODE'):
            wp_von = wahlperiode.find('MDBWP_VON').text
            wp_bis = wahlperiode.find('MDBWP_BIS').text

            # Überprüfen, ob die Wahlperiode 2024 aktiv ist
            if wp_von and wp_bis:  # Sicherstellen, dass die Werte nicht None sind
                if wp_von <= current_year_end and wp_bis >= current_year_start:
                    # Attribute des Abgeordneten sammeln
                    mdb_data = {
                        'ID': mdb.find('ID').text,
                        'NAME': [name.text for name in mdb.findall('NAMEN/NAME')],
                        'GEBURTSDATUM': bio_info.find('GEBURTSDATUM').text if bio_info.find('GEBURTSDATUM') is not None else None,
                        'GEBURTSORT': bio_info.find('GEBURTSORT').text if bio_info.find('GEBURTSORT') is not None else None,
                        'GEBURTSLAND': bio_info.find('GEBURTSLAND').text if bio_info.find('GEBURTSLAND') is not None else None,
                        'GESCHLECHT': bio_info.find('GESCHLECHT').text if bio_info.find('GESCHLECHT') is not None else None,
                        'FAMILIENSTAND': bio_info.find('FAMILIENSTAND').text if bio_info.find('FAMILIENSTAND') is not None else None,
                        'RELIGION': bio_info.find('RELIGION').text if bio_info.find('RELIGION') is not None else None,
                        'BERUF': bio_info.find('BERUF').text if bio_info.find('BERUF') is not None else None,
                        'PARTEI_KURZ': bio_info.find('PARTEI_KURZ').text if bio_info.find('PARTEI_KURZ') is not None else None,
                        'VITA_KURZ': bio_info.find('VITA_KURZ').text if bio_info.find('VITA_KURZ') is not None else None,
                        'VEROEFFENTLICHUNGSPFLICHTIGES': bio_info.find('VEROEFFENTLICHUNGSPFLICHTIGES').text if bio_info.find('VEROEFFENTLICHUNGSPFLICHTIGES') is not None else None,
                        'WAHLPERIODE': {
                            'WP': wahlperiode.find('WP').text,
                            'MDBWP_VON': wp_von,
                            'MDBWP_BIS': wp_bis,
                            'WKR_NUMMER': wahlperiode.find('WKR_NUMMER').text,
                            'WKR_NAME': wahlperiode.find('WKR_NAME').text,
                            'WKR_LAND': wahlperiode.find('WKR_LAND').text,
                            'LISTE': wahlperiode.find('LISTE').text,
                            'MANDATSART': wahlperiode.find('MANDATSART').text,
                        }
                    }
                    active_mdbs.append(mdb_data)

# Ausgabe der aktiven Abgeordneten
for mdb in active_mdbs:
    print(mdb)


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [2]:
mdb

{'ID': '11005306',
 'NAME': ['\n        '],
 'GEBURTSDATUM': '15.03.1994',
 'GEBURTSORT': 'Piatra Neamț',
 'GEBURTSLAND': 'Rumänien',
 'GESCHLECHT': 'weiblich',
 'FAMILIENSTAND': 'ledig',
 'RELIGION': None,
 'BERUF': 'Kulturwissenschaftlerin',
 'PARTEI_KURZ': 'SPD',
 'VITA_KURZ': None,
 'VEROEFFENTLICHUNGSPFLICHTIGES': None,
 'WAHLPERIODE': {'WP': '20',
  'MDBWP_VON': '16.05.2023',
  'MDBWP_BIS': '04.03.2024',
  'WKR_NUMMER': None,
  'WKR_NAME': None,
  'WKR_LAND': None,
  'LISTE': 'BE',
  'MANDATSART': 'Landesliste'}}

In [3]:
import pandas as pd

# In einen DataFrame umwandeln
df_mdb = pd.DataFrame(active_mdbs)
print(df_mdb.head())  # Zeigt nur die ersten paar Zeilen an


         ID          NAME GEBURTSDATUM GEBURTSORT GEBURTSLAND GESCHLECHT  \
0  11000001  [\n        ]   20.10.1930  Stuttgart        None   männlich   
1  11000001  [\n        ]   20.10.1930  Stuttgart        None   männlich   
2  11000001  [\n        ]   20.10.1930  Stuttgart        None   männlich   
3  11000001  [\n        ]   20.10.1930  Stuttgart        None   männlich   
4  11000001  [\n        ]   20.10.1930  Stuttgart        None   männlich   

   FAMILIENSTAND    RELIGION  \
0  keine Angaben  katholisch   
1  keine Angaben  katholisch   
2  keine Angaben  katholisch   
3  keine Angaben  katholisch   
4  keine Angaben  katholisch   

                                               BERUF PARTEI_KURZ VITA_KURZ  \
0  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...         CDU      None   
1  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...         CDU      None   
2  Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...         CDU      None   
3  Rechtsanwalt, Wirtschaftsprüfer, Un

In [4]:
df_mdb

,ID,NAME,GEBURTSDATUM,GEBURTSORT,GEBURTSLAND,GESCHLECHT,FAMILIENSTAND,RELIGION,BERUF,PARTEI_KURZ,VITA_KURZ,VEROEFFENTLICHUNGSPFLICHTIGES,WAHLPERIODE
0,11000001,[\n ],20.10.1930,Stuttgart,None,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,"{'WP': '5', 'MDBWP_VON': '19.10.1965', 'MDBWP_..."
1,11000001,[\n ],20.10.1930,Stuttgart,None,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,"{'WP': '6', 'MDBWP_VON': '20.10.1969', 'MDBWP_..."
2,11000001,[\n ],20.10.1930,Stuttgart,None,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,"{'WP': '7', 'MDBWP_VON': '13.12.1972', 'MDBWP_..."
3,11000001,[\n ],20.10.1930,Stuttgart,None,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,"{'WP': '8', 'MDBWP_VON': '14.12.1976', 'MDBWP_..."
4,11000001,[\n ],20.10.1930,Stuttgart,None,männlich,keine Angaben,katholisch,"Rechtsanwalt, Wirtschaftsprüfer, Universitätsp...",CDU,None,None,"{'WP': '9', 'MDBWP_VON': '04.11.1980', 'MDBWP_..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11667,11005179,[\n ],04.07.1965,Wehrda (jetzt Marburg),None,männlich,"verheiratet, 2 Kinder",evangelisch,"Arzt, Facharzt für Chirurgie",SPD,None,None,"{'WP': '20', 'MDBWP_VON': '26.10.2021', 'MDBWP..."
11668,11005218,[\n ],03.06.1968,Arolsen,None,männlich,"geschieden, 2 Kinder",evangelisch,Oberstudienrat,CDU,None,None,"{'WP': '20', 'MDBWP_VON': '26.10.2021', 'MDBWP..."
11669,11005227,[\n ],27.10.1982,Frankfurt am Main,None,weiblich,"verheiratet, 3 Kinder",None,Lehrerin,BÜNDNIS 90/DIE GRÜNEN,None,None,"{'WP': '20', 'MDBWP_VON': '26.10.2021', 'MDBWP..."
11670,11005233,[\n ],19.05.1970,Karlsruhe,None,weiblich,1 Kind,evangelisch,Bürgermeisterin,CDU,None,None,"{'WP': '20', 'MDBWP_VON': '26.10.2021', 'MDBWP..."
